# ASMR Preprocessing
## Batch script

Should be a (roughly identical) copy of the .py script of the same name. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 72

# from os import path, mkdir
import os
import mne
from mne.viz.ica import _prepare_data_ica_properties
mne.set_log_level('error')
from scipy.stats import zscore
from autoreject import Ransac, get_rejection_threshold, AutoReject

### Set parameters

# PERMANENTLY EXCLUDE 'ASMR_08' & ASMR_14 & 'ASMR_18' because they had no reports of ASMR that lasted > 1 s each
# Likewise, ASMR_01 and ASMR_20 have 0-1 ASMR events after preprocessing
subjects = ['ASMR_02', 'ASMR_03', 'ASMR_05', 
            'ASMR_07', 
            'ASMR_09',
            'ASMR_10', 'ASMR_11',
            'ASMR_13','ASMR_16', 'ASMR_17', 
            'ASMR_19', 
            'ASMR_21', 'ASMR_22',
           ]

# for all multi-threaded operations
n_jobs = 6

# Filter settings
l_freq = 0.1
h_freq = 50.0
l_freq_ica = 1.
l_trans_bandwidth = 'auto'
h_trans_bandwidth = 'auto'
filter_length='auto'
filter_method = 'fir'
filter_picks = ['eeg', 'eog', 'bio']

rej_log_list = []

# ICA settings
ica_random_state = 42  # seed so ICA is reproducable each time it's run
n_components = .995     # Specify n_components as a decimal to set % explained variance
ica_zthresh = 3.291

# Events
# Note: ASMRcontrol trials and codes will be defined later
event_id = {'video1':1, 'video2':2,
            'video3':3, 'video4':4,
            'video5':5, 'video6':6,
            'rest_start':7, 'rest':8,
            'video_end':25,
            'ASMRbutton/start':64, 'ASMR': 65, 'ASMRbutton/end':128,
            'ASMRctrl':66
           }


cond_of_interest = sorted(['ASMRstart','ASMRcontrol','rest'])

rest_duration = 30 * 1000 # convert s to ms
rest_interval =  1 * 1000

# Epoching settings
tmin = -1.0  # start of each epoch (in sec)
tmax =  1.0  # end of each epoch (in sec)
baseline = None  #(None, 0)
detrend = 0
reject = None
flat = None

# standard montage file to look up channel locations
montage_fname = 'standard_1005'

out_path = '../'
epo_path = out_path + 'Epochs/'

if not os.path.exists(out_path):
    os.mkdir(out_path)
if not os.path.exists(epo_path):
    os.mkdir(epo_path)

for subject in subjects:
    print('\n-------------------------')
    print('-------- ' + subject + ' --------')
    print('-------------------------')
    ### Subject-specific paramters
    # Where to find and save files
    data_path = '../../EEG data/' + subject + '/'
    log_path = out_path + 'preprocessing logs/' + subject + '/'
    if not os.path.exists(log_path):
        os.mkdir(log_path)
    else:
        for filename in os.listdir(log_path):
            file_path = os.path.join(log_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

                
    # output file names - set to follow MNE conventions
    epochs_fname  = epo_path + subject + '-epo.fif'
    # psd_data_fname = csv_path + subject + '-psd.csv'
    asmr_reports_fname = log_path + subject + '_ASMR_reports.csv'

    ### Subject-specific hacks
    if subject=='ASMR_20':
        n_components = .9999
        
    ### Import data
    raw_fname = data_path + subject + '.vhdr'
    raw = mne.io.read_raw_brainvision(raw_fname,
                                        preload=True,
                                       )
    raw.set_channel_types({'HEOG':'eog','VEOG':'eog','GSR':'bio'})
    raw.set_montage(montage_fname)

    
    ### Event processing #######################################################################
    events, event_dict = mne.events_from_annotations(raw)

    ##################################
    # CODE PROCESSING
    ##################################
    ## Create epochs from rest period

    r_starts = np.squeeze(np.where(events[:, 2] == event_id['rest_start']))
    trial_ends = np.squeeze(np.where(events[:, 2] == event_id['video_end']))
    # trial_ends include both rest blocks and ASMR videos; find only ends of rest blocks
    r_ends = np.squeeze(trial_ends[np.searchsorted(trial_ends, r_starts)])

    r_idx = np.column_stack((r_starts, r_ends))
    r_intvl = events[r_idx, 0]

    r_segs = []
    for i in r_intvl:
        r_segs.append(np.arange(i[0], i[1]-rest_interval, rest_interval))

    r_segs = np.concatenate(r_segs)    
    rest_events = np.transpose([r_segs, 
                                np.tile(0, len(r_segs)), 
                                np.tile(event_id['rest'], len(r_segs))
                               ])

    ####################
    ## Find ASMR AND CONTROL events

    asmr_events = []
    ctrl_events = []

    # find events marking start of ASMR videos
    v_starts = np.squeeze(np.where((events[:, 2] > 0)  & (events[:, 2] < 7)))
    trial_ends = np.where(events[:, 2] == event_id['video_end'])[0]
    # trial_ends include both control and ASMR videos; find only ends of ASMR videos
    v_ends = trial_ends[np.searchsorted(trial_ends, v_starts)]

    for v_num, v_start_idx in np.ndenumerate(v_starts):
        v_end_idx = v_ends[v_num] + 1
        # create array of codes in this video, INCLUDING video start code, and including video end code
        this_vid = events[v_start_idx:v_end_idx, :]
        vid_num = this_vid[0, 2]

        # find start of ASMR periods
        asmr_starts = np.squeeze(np.where((this_vid[:, 2] == event_id['ASMRbutton/start']) ))
        # find instances of two 'start' button presses in a row
        dbl_starts = asmr_starts[this_vid[asmr_starts-1, 2] == event_id['ASMRbutton/start']]
        # remove the doubled rows
        this_vid = np.delete(this_vid, dbl_starts, axis=0)

        asmr_ends = np.squeeze(np.where((this_vid[:, 2] == event_id['ASMRbutton/end'])))
        dbl_ends = asmr_ends[this_vid[asmr_ends-1, 2] == event_id['ASMRbutton/end']]
        # remove the doubled rows
        this_vid = np.delete(this_vid, dbl_ends, axis=0)
        # update 
        asmr_starts = np.squeeze(np.where(this_vid[:, 2] == event_id['ASMRbutton/start']))
        asmr_ends = np.squeeze(np.where((this_vid[:, 2] == event_id['ASMRbutton/end']) | (this_vid[:, 2] == event_id['video_end'])))

        ctrl_starts = np.squeeze(np.where((this_vid[:, 2] == event_id['ASMRbutton/end']) | (this_vid[:, 2] == vid_num)))
        ctrl_ends   = np.squeeze(np.where((this_vid[:, 2] == event_id['ASMRbutton/start']) | (this_vid[:, 2] == event_id['video_end'])))

        if (asmr_starts.ndim == 0):
            asmr_starts = np.reshape(asmr_starts, 1)

        if asmr_starts.size != 0:     

            if this_vid[-2, 2] != event_id['ASMRbutton/start']:
                asmr_ends = asmr_ends[:-1]

            asmr_durations = this_vid[asmr_ends, 0] - this_vid[asmr_starts, 0]
            asmr_seg_len = asmr_durations // 1000

            for idx in range(len(asmr_starts + 1)):
                event = this_vid[asmr_starts[idx]]

                for seg in range(asmr_seg_len[idx]):
                    asmr_events.append([event[0] + (seg * 1000), 
                                       0, 
                                       event_id['ASMR']
                                      ])
            # remove last "end" if video ended during ASMR period. Only needed if ASMR happened
            if(len(ctrl_ends) > len(ctrl_starts)):
                ctrl_ends = ctrl_ends[:-1]

        ctrl_durations = this_vid[ctrl_ends, 0] - this_vid[ctrl_starts, 0]
        ctrl_seg_len = ctrl_durations // 1000

        if ctrl_starts.ndim == 0:
            ctrl_starts = ctrl_starts.reshape([1])
            ctrl_ends = ctrl_ends.reshape([1])
            ctrl_seg_len = ctrl_seg_len.reshape([1])



        for idx in range(len(ctrl_starts + 1)):
            event = this_vid[ctrl_starts[idx]]

            for seg in range(ctrl_seg_len[idx]):
                ctrl_events.append([event[0] + (seg * 1000), 
                                   0, 
                                   event_id['ASMRctrl']
                                  ])


    asmr_events = np.asarray(asmr_events)    
    ctrl_events = np.asarray(ctrl_events)         
    ## Combine into new events array
    events_new = np.vstack((rest_events, asmr_events, ctrl_events))
    # sort by time
    events_new = events_new[events_new[:, 0].argsort()] 
    mne.write_events(log_path + subject + '_events.tsv', events_new)

    # Reduce event_id to events retained after rejection
    event_id_new = {}
    for key, value in event_id.items():
        if np.isin(value, np.unique(events_new[:, 2])):
            event_id_new[key] = value

    # export plot of events
    matplotlib.rcParams['lines.markersize'] = 2
    matplotlib.rcParams['figure.figsize'] = (45, 3)
    fig = mne.viz.plot_events(events_new, raw.info['sfreq'], event_id=event_id_new, show=False)  
    fig.savefig(log_path + subject + '_event_timing.png');
    plt.close(fig)
    # End of event code processing section #####
    ############################################
    
    ### Data Cleaning
    # Filter for ICA
    raw_ica = raw.copy().filter(l_freq_ica, h_freq,
                            l_trans_bandwidth = l_trans_bandwidth,
                            h_trans_bandwidth = h_trans_bandwidth,
                            filter_length=filter_length,
                            method='fft',
                            picks = mne.pick_types(raw.info, eeg=True, eog=True),
                            n_jobs = n_jobs)

    # Filter for final
    raw_filt = raw.copy().filter(l_freq, h_freq,
                                 l_trans_bandwidth = l_trans_bandwidth,
                                 h_trans_bandwidth = h_trans_bandwidth,
                                 filter_length=filter_length,
                                 method=filter_method,
                                 picks = filter_picks,
                                 n_jobs = n_jobs)

    fig = raw_filt.plot_psd(fmax=80, n_jobs=n_jobs, average=False, spatial_colors=True, show=False);
    fig.savefig(log_path + subject + '_filtered_psd.png');
    plt.close(fig)

    # Break raw data into 1 s epochs for autoreject
    tstep = 1.0
    events_ica = mne.make_fixed_length_events(raw_ica, duration=tstep)
    epochs_ica = mne.Epochs(raw_ica, events_ica,
                            tmin=0.0, tmax=tstep,
                            baseline=None,
                            preload=True)
    ### Find appropriate rejection threshold to eliminate noisy trials from ICA fitting
    reject = get_rejection_threshold(epochs_ica)

    # Find and mark bad channels to be ignored by ICA
    ransac = Ransac(n_jobs=n_jobs, random_state=ica_random_state, verbose=False)
    ransac.fit(epochs_ica)
    epochs_ica.info['bads'] = ransac.bad_chs_

    # Fit ICA
    ica = mne.preprocessing.ICA(n_components=n_components,
                                random_state=ica_random_state,
                                max_iter='auto')
    ica.fit(epochs_ica,
            reject=reject,
            tstep=tstep)

    fig = ica.plot_components(show=False);
    for f_idx, f in enumerate(fig):
        f.savefig(log_path + subject + '_ica_topomaps' + str(f_idx) + '.png');
        plt.close(f)
        
    # Identify ocular ICs
    # At least for our ASMR dataset, the default *z* threshold doesn't work for
    # all subjects. This routine starts with the default (*z* = 3) and steps down
    # until at least 2 EOG components are identified.
    # The issue with this is that it assumes there will always be at least 2 EOG
    # components (blinks are always present, but horizontal movements are not
    # always present), and may not work if there are > 3 components, if the
    # score of the third is > `z_step` less than the score of the second.
    ica.exclude = []
    num_excl = 0
    z_thresh = 3.0
    z_step = 0.2 # decrement zthresh by this much each time

    while num_excl < 2:
        eog_indices, eog_scores = ica.find_bads_eog(raw_ica,threshold=z_thresh)
        num_excl = len(eog_indices)
        z_thresh -= z_step # won't impact things if num_excl is 2 or more

    ica.exclude = eog_indices
    z_thresh_final = round(z_thresh + z_step, 2)

    # Find components where at least 1 electrode has a z score > ica_zthresh (conventionally z ≤ 2.96, p < .05)
    comp_dat = ica.get_components()
    ica_zscores = abs(zscore(comp_dat, axis=1))   
    outlier_comps = np.where([max(ica_zscores[:, i]) > ica_zthresh for i in range(ica_zscores.shape[1])])[0].tolist()
    ica.exclude.extend(outlier_comps)
    ica_zscores = pd.DataFrame(ica_zscores)
    ica_zscores.to_csv(log_path + subject + 'ica_zscores.csv')

    # Manual removal/re-addition of ICs based on visual inspection
    # (this is a kluge until we have a better automated IC classification system)
    if subject == 'ASMR_10':
        ica.exclude.remove(10)
    if subject == 'ASMR_19':
        for ic in [4, 6]:
            ica.exclude.remove(ic)
    if subject == 'ASMR_21':
        for ic in [7]:
            ica.exclude.remove(ic)
    if subject == 'ASMR_22':
        for ic in [3, 5, 10, 11, 22]:
            ica.exclude.remove(ic)

    # barplot of ICA component "EOG match" scores
    fig = ica.plot_scores(eog_scores, show=False);
    fig.savefig(log_path + subject + '_ica_score.png');
    plt.close(fig)

    # plot diagnostics
    for i in ica.exclude:
        fig = ica.plot_properties(raw_ica, picks=i, psd_args={'fmax': h_freq}, show=False);
        fig[0].savefig(log_path + subject + '_ica_eog_' + str(i) + '.png');
        plt.close(fig[0])

    #### Segment filtered raw data into epochs for final analysis
    epochs = mne.Epochs(raw_filt,
                        events_new, event_id_new,
                        tmin, tmax,
                        baseline=baseline, detrend=detrend,
                        reject=reject, flat=None,
                        preload=True
                       )

    # Apply ICA correction to epochs
    epochs_postica = ica.apply(epochs.copy())
    epochs_postica.info['bads'] = ransac.bad_chs_
    epochs_postica = epochs_postica.interpolate_bads()

    # Apply AutoReject to clean epochs
    ar = AutoReject(n_jobs=n_jobs, random_state=ica_random_state, verbose=False)
    epochs_clean = ar.fit_transform(epochs_postica)
    # Re-reference to average of all channels, now that they are cleaned
    epochs_clean.set_eeg_reference(ref_channels='average');

    # Report on how much was rejected
    rm_epochs = epochs_postica.selection.shape[0] - epochs_clean.selection.shape[0]
    pct_epochs = rm_epochs / epochs_postica.selection.shape[0] * 100
    print('n epochs removed: ' + str(rm_epochs)
          + '\nrepresenting ' + str(round(pct_epochs, 2)) + ' % of data')

    # Save log of rejections
    rej_log_list.append(pd.DataFrame({'Subject':subject,
                                      'n Trials Rej':rm_epochs,
                                      '% Trials Rej':round(pct_epochs, 2),
                                      'n Chans Fixed':len(epochs_ica.info['bads']),
                                      'n ICs removed':len(ica.exclude)
                                      }, index=[0])
                         )



    # Save cleaned epochs
    epochs_clean.save(epochs_fname, overwrite=True)

    fig = epochs_clean.average().plot(spatial_colors=True, show=False);
    fig.savefig(log_path + subject + '_epochs_avg.png');
    plt.close(fig)

rej_log = pd.concat(rej_log_list, ignore_index=True)
rej_log.to_csv(log_path + 'ASMR_rej_log.csv')


-------------------------
-------- ASMR_02 --------
-------------------------
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    9.2s remaining:   18.3s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    9.2s remaining:    9.2s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    9.2s remaining:    4.6s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    9.3s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    9.3s finished


n epochs removed: 0
representing 0.0 % of data
Overwriting existing file.

-------------------------
-------- ASMR_03 --------
-------------------------
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    8.1s remaining:   16.2s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    8.1s remaining:    8.1s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    8.2s remaining:    4.1s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    8.2s finished


n epochs removed: 21
representing 7.72 % of data
Overwriting existing file.

-------------------------
-------- ASMR_05 --------
-------------------------
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


...

n epochs removed: 1
representing 2.22 % of data
Overwriting existing file.

-------------------------
-------- ASMR_19 --------
-------------------------
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    8.5s remaining:   17.0s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    8.5s remaining:    8.5s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    8.5s remaining:    4.3s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    8.6s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    8.6s finished


n epochs removed: 0
representing 0.0 % of data
Overwriting existing file.

-------------------------
-------- ASMR_21 --------
-------------------------
Estimating rejection dictionary for eeg
Estimating rejection dictionary for eog


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    9.1s remaining:   18.3s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    9.2s remaining:    9.2s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    9.2s remaining:    4.6s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    9.3s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    9.3s finished


ValueError: list.remove(x): x not in list

### Variance


In [ ]:
picks = range(ica.n_components_)
kind, dropped_indices, epochs_src, data = _prepare_data_ica_properties(
    epochs_ica, ica)
ica_data = np.swapaxes(data[:, picks, :], 0, 1)
dropped_src = ica_data

# fig, ax = plt.subplots(ica.n_components_, 1)
for idx in picks:
    epoch_var = np.var(ica_data[idx], axis=1)
    drop_var = np.var(dropped_src[idx], axis=1)
    drop_indices_corrected = \
        (dropped_indices -
         np.arange(len(dropped_indices))).astype(int)
    epoch_var = np.insert(arr=epoch_var,
                          obj=drop_indices_corrected,
                          values=drop_var[dropped_indices],
                          axis=0)
    plt.hist(zscore(epoch_var), density=True)
    plt.title('IC ' + str(idx))
    plt.yscale('log')
    plt.show()